In [1]:
import ee
# Authenticate and initialize Earth Engine
ee.Authenticate()

# Insert your own folder name
ee.Initialize(project = "ee-kuzeyuvercinka")

In [3]:

# Define the region of interest (Frankfurt, Germany)
frankfurt = ee.Geometry.Polygon([
    [[8.5268, 50.1882], [8.5268, 50.2333], [8.7352, 50.2333], [8.7352, 50.1882]]
])

# Load Sentinel-2 surface reflectance imagery
s2 = ee.ImageCollection('COPERNICUS/S2_SR') \
    .filterBounds(frankfurt) \
    .filterDate('2023-05-01', '2023-09-30') \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))

# Function to calculate NDVI and add it as a band
def add_ndvi(image):
    ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')  # NIR and Red bands
    return image.addBands(ndvi)

s2_with_ndvi = s2.map(add_ndvi)

# Get median NDVI and true-color composite for the selected time period
ndvi_median = s2_with_ndvi.select('NDVI').median().clip(frankfurt)
true_color_median = s2.median().clip(frankfurt)

# Apply thresholds for land cover classification
green_space = ndvi_median.gt(0.3).selfMask()  # NDVI > 0.3 for vegetation
water = ndvi_median.lt(0).selfMask()          # NDVI < 0 for water bodies
built_up = ndvi_median.gt(-0.05).And(ndvi_median.lte(0.3)).selfMask()  # -0.05 <= NDVI <= 0.3

# Calculate total green space area
pixel_area = green_space.multiply(ee.Image.pixelArea())
green_area = pixel_area.reduceRegion(
    reducer=ee.Reducer.sum(),
    geometry=frankfurt,
    scale=10,
    maxPixels=1e9
)

# Calculate water and built-up area
water_area = water.multiply(ee.Image.pixelArea()).reduceRegion(
    reducer=ee.Reducer.sum(),
    geometry=frankfurt,
    scale=10,
    maxPixels=1e9
)
built_up_area = built_up.multiply(ee.Image.pixelArea()).reduceRegion(
    reducer=ee.Reducer.sum(),
    geometry=frankfurt,
    scale=10,
    maxPixels=1e9
)

# Print results
print('Total green space area (m²):', green_area.getInfo()['NDVI'])
print('Total water area (m²):', water_area.getInfo()['NDVI'])
print('Total built-up area (m²):', built_up_area.getInfo()['NDVI'])

# Visualization Parameters
true_color_vis_params = {
    'bands': ['B4', 'B3', 'B2'],  # Red, Green, Blue
    'min': 0,
    'max': 3000,
    'gamma': 1.3
}

ndvi_vis_params = {
    'min': -0.1,
    'max': 0.8,
    'palette': ['brown', 'yellow', 'green']
}

green_space_vis_params = {
    'palette': ['#00FF00'],  # Bright green for vegetation
    'opacity': 1
}

water_vis_params = {
    'palette': ['#0000FF'],  # Bright blue for water bodies
    'opacity': 1
}

built_up_vis_params = {
    'palette': ['#FFA500'],  # Orange for built-up areas
    'opacity': 1
}

# Export True Color Image (Realistic RGB)
task_true_color = ee.batch.Export.image.toDrive(
    image=true_color_median.visualize(**true_color_vis_params),
    description='TrueColor_Frankfurt_new2',
    scale=10,
    region=frankfurt,
    fileFormat='GeoTIFF',
    maxPixels=1e9
)
task_true_color.start()

# Export NDVI Image (Vegetation Index)
task_ndvi = ee.batch.Export.image.toDrive(
    image=ndvi_median.visualize(**ndvi_vis_params),
    description='NDVI_Frankfurt_new2',
    scale=10,
    region=frankfurt,
    fileFormat='GeoTIFF',
    maxPixels=1e9
)
task_ndvi.start()

# Export Green Space Image
task_green_space = ee.batch.Export.image.toDrive(
    image=green_space.visualize(**green_space_vis_params),
    description='GreenSpace_Frankfurt_new2',
    scale=10,
    region=frankfurt,
    fileFormat='GeoTIFF',
    maxPixels=1e9
)
task_green_space.start()

# Export Water Image
task_water = ee.batch.Export.image.toDrive(
    image=water.visualize(**water_vis_params),
    description='Water_Frankfurt_new2',
    scale=10,
    region=frankfurt,
    fileFormat='GeoTIFF',
    maxPixels=1e9
)
task_water.start()

# Export Built-Up Area Image
task_built_up = ee.batch.Export.image.toDrive(
    image=built_up.visualize(**built_up_vis_params),
    description='BuiltUp_Frankfurt_new2',
    scale=10,
    region=frankfurt,
    fileFormat='GeoTIFF',
    maxPixels=1e9
)
task_built_up.start()

print("Export tasks for distinct images started!")


Total green space area (m²): 42741197.92735822
Total water area (m²): 31048.163081180348
Total built-up area (m²): 31884483.58328813
Export tasks for distinct images started!
